In [1]:
# Dependencies and Setup
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import tkinter
import os
import gmaps
import math

# Google developer API key
from config import gkey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import api_key

# File to Load (Remember to change these)
analysis_data_to_load = "output_data/TempVsCrime_2015.csv"

In [17]:
#lets define all Variables we need
cities = ['Los Angeles','Chicago','Vancouver, CA','Philadelphia','New York']
cities_selected=[]
lat =[]
lon =[]

#define units we want
units = "metric"

#loop through all cities in our list: cities
for idx, city in enumerate(cities):    
    # Save config information and Build query URL
    url = "http://api.openweathermap.org/data/2.5/weather?"    
    query_url = f"{url}appid={api_key}&units={units}&q={city}"
    
    #check to see if the reponse has code 200. if it does not skip it
    if requests.get(query_url).status_code == 200:
        print (f'Processing Record {idx+1} of Set of {len(cities)} FOUND & ADDED: |{city}')
        response = requests.get(query_url).json()
        city
        cities_selected.append(city)
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
    else:
        print (f'Processing Record {idx+1} of Set of {len(cities)} NOT FOUND: |{city}')

cities_dict = {
    "City": cities_selected,
    "Lat": lat,
    "Lng":lon
      }

cities_data = pd.DataFrame(cities_dict)
cities_data.set_index('City',inplace=True)

cities_data.rename(index={'Vancouver, CA':'Vancouver'}, inplace=True)
cities_data

Processing Record 1 of Set of 5 FOUND & ADDED: |Los Angeles
Processing Record 2 of Set of 5 FOUND & ADDED: |Chicago
Processing Record 3 of Set of 5 FOUND & ADDED: |Vancouver, CA
Processing Record 4 of Set of 5 FOUND & ADDED: |Philadelphia
Processing Record 5 of Set of 5 FOUND & ADDED: |New York


,Lat,Lng
City,,
Los Angeles,34.05,-118.24
Chicago,41.88,-87.62
Vancouver,49.26,-123.12
Philadelphia,39.95,-75.16
New York,40.73,-73.99


In [18]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}

# Create the Layout
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to cities
markers = gmaps.symbol_layer(cities_data[['Lat', 'Lng']], fill_color='green', stroke_color='green', scale=5)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='2px', wi…

In [19]:
analysis_df = pd.read_csv(analysis_data_to_load)
analysis_df_crime = analysis_df.drop(columns= ['Los Angeles','Chicago','Vancouver','Philadelphia','New York', 'Temp Diff (NY-PHI)'])
analysis_df_crime

,Crime Los Angeles,Crime Chicago,Crime New York,Crime Philadelphia,Crime Vancouver
0,17981,20656,34533,13634,2678
1,15930,16287,31540,12266,2328
2,17907,21560,38023,14367,2440
3,16869,21610,39846,16766,2515
4,17839,23570,42321,17067,2839
5,17423,23059,41662,16429,2859
6,18553,24101,44379,17227,3224
7,18877,24685,45096,17388,2937
8,18152,22996,41942,15358,2876
9,19065,22979,41250,15188,3216


In [20]:
analysis_df_crime_sum = pd.DataFrame(analysis_df_crime.sum(axis = 0, skipna = True))
analysis_df_crime_sum.columns = ['Total Crimes']
analysis_df_crime_sum.rename(index={'Crime Los Angeles':'Los Angeles',
                                    'Crime Chicago':'Chicago',
                                    'Crime New York':'New York',
                                    'Crime Philadelphia':'Philadelphia',
                                    'Crime Vancouver':'Vancouver'}, inplace=True)
analysis_df_crime_sum.index.name = 'City'
analysis_df_crime_sum

,Total Crimes
City,
Los Angeles,213910
Chicago,262995
New York,478578
Philadelphia,183300
Vancouver,34328


In [21]:
cities_data['Crimes'] = analysis_df_crime_sum['Total Crimes']
cities_data

,Lat,Lng,Crimes
City,,,
Los Angeles,34.05,-118.24,213910
Chicago,41.88,-87.62,262995
Vancouver,49.26,-123.12,34328
Philadelphia,39.95,-75.16,183300
New York,40.73,-73.99,478578


In [23]:
figure_layout = {
#     'width': '3000px',
#     'height': '2000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Create a max temp Heatmap layer
fig = gmaps.figure(figure_layout)

#Create zone Lat Long and Heat
heat_layer = gmaps.heatmap_layer(cities_data[['Lat', 'Lng']], weights=cities_data['Crimes'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 300000
heat_layer.point_radius = 1.5

fig.add_layer(heat_layer)
markers = gmaps.symbol_layer(cities_data[['Lat', 'Lng']], fill_color='green', stroke_color='green', scale=1)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
analysis_df_temp = analysis_df.drop(columns= ['Crime Los Angeles','Crime Chicago','Crime Vancouver','Crime Philadelphia','Crime New York', 'Temp Diff (NY-PHI)'])
analysis_df_temp

,Los Angeles,Chicago,Vancouver,Philadelphia,New York
0,49.148462,22.481637,42.722406,26.132572,24.476729
1,52.858782,13.211865,45.614484,20.201421,18.971325
2,58.036861,31.705260,45.869559,34.277820,33.833749
3,56.350911,44.920098,46.403030,53.217411,50.321019
4,58.175460,54.258154,57.393804,67.803094,65.718317
5,69.791577,61.250251,64.134975,71.651405,68.624498
6,73.210653,71.785134,68.455081,74.954760,75.644339
7,76.715166,70.811460,66.142129,73.707688,75.277758
8,76.246112,68.263577,57.173755,69.869135,70.857912
9,71.209255,54.735544,53.661417,54.136649,54.776640


In [25]:
analysis_df_temp_avg = pd.DataFrame(analysis_df_temp.mean(axis = 0, skipna = True))
analysis_df_temp_avg.columns = ['Temp Avg']
analysis_df_temp_avg.index.name = 'City'
analysis_df_temp_avg

,Temp Avg
City,
Los Angeles,62.887782
Chicago,48.197740
Vancouver,52.190413
Philadelphia,53.682487
New York,53.024414


In [26]:
cities_data['Temp Avg'] = analysis_df_temp_avg['Temp Avg']
cities_data

,Lat,Lng,Crimes,Temp Avg
City,,,,
Los Angeles,34.05,-118.24,213910,62.887782
Chicago,41.88,-87.62,262995,48.197740
Vancouver,49.26,-123.12,34328,52.190413
Philadelphia,39.95,-75.16,183300,53.682487
New York,40.73,-73.99,478578,53.024414


In [27]:
figure_layout = {
#     'width': '3000px',
#     'height': '2000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Create a max temp Heatmap layer
fig = gmaps.figure(figure_layout)

#Create zone Lat Long and Heat
heat_layer = gmaps.heatmap_layer(cities_data[['Lat', 'Lng']], weights=cities_data['Temp Avg'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.5)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 62
heat_layer.point_radius = 1.5

fig.add_layer(heat_layer)
markers = gmaps.symbol_layer(cities_data[['Lat', 'Lng']], fill_color='green', stroke_color='green', scale=1)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))